In [1]:
#hide
%load_ext autoreload
%autoreload 2

In [2]:
#hide
from blobster import *

In [3]:
# pydyverse
import json
import os
import glob
import io
import re
import zipfile
from datetime import date
from datetime import datetime
from functools import wraps
from pathlib import Path
import logging
import numpy as np
import pandas as pd
import pandas_profiling
import pandas_flavor as pf
import jdc

class Datapipe:
    def __init__(self, service=None, entity=None, meta=None, runtime=None):
        self.service: str = service
        self.runtime: str = runtime
        self.entity: str = entity
        self.meta = meta
        self.today = datetime.today().strftime("%Y-%m-%d")
        self.profile_report = None

datapipe = Datapipe()

In [4]:
#blobster
azure_blob_storage = AzureBlobStorage(account='vfudevblobstorage',
                                      key='eEbmjhexKPoGy+e8B2aUiDBPIG0qRThKRenJLgVbGN2TlcwMty9NopeUmwzIQ67DpYvcr8AwgPjobAiT0Zmzkw==')
azure_blob_storage.connect()

In [5]:
#wf
datapipe.service = 'vfu'
datapipe.runtime = 'local'

In [6]:
meta

NameError: name 'meta' is not defined

### items

In [ ]:
#vfu
@pf.register_dataframe_method
def step_consolidate_item_learning_levels(df, col, mapping=None):
    if mapping:
        learning_level_names = mapping
    else:
        learning_level_names = {'1': '1:Core', '2':'2:Intermediate', '3':'3:Advanced',
                    '4':'4:Expert', 'SCM Essentials':'1:Core', '5':'5:Others'}
    df[col] = df[col].str.split(' ').str[0]
    df[col] = np.where(df[col] == '', 'Missing Info', df[col])
    df[col] = df[col].map(learning_level_names)  
    df[col] = df[col].fillna(value='Missing Info')
    return df

In [ ]:
#wf
datapipe.entity = 'assets'

In [ ]:
#pydyverse
data_definition = None
#pydyverse
@pf.register_dataframe_method
def load_data_definition(entity):
    global data_definition
    data_definition = azure_blob_storage.blob_to_df(container_name=f"{datapipe.service}-data-definitions",
                                                        blob_name=f"{datapipe.service}-data-definition.xlsx",)
    #TODO: to be fixed
    data_definition.unique_identifier = data_definition.unique_identifier.astype('bool')
    data_definition = data_definition[data_definition.entity == entity]

In [ ]:
load_data_definition(entity=datapipe.entity)

In [ ]:
data_definition

In [ ]:
#vfu
@pf.register_dataframe_method
def step_load_entity_definition(df, all_entities=False):
    """loads data definition of entity"""
    datapipe.meta = azure_blob_storage.blob_to_df(container_name=f"{datapipe.service}-data-definitions",
                                                        blob_name=f"{datapipe.service}-data-definition.xlsx",)
    if not all_entities:
        datapipe.meta = datapipe.meta[datapipe.meta.entity == datapipe.entity]
    return df

In [ ]:
#vfu
@pf.register_dataframe_method
def step_load_raw_data(df):
    df = azure_blob_storage.blobs_to_df(
        container_name=f"{datapipe.service}-{datapipe.entity}-raw"
    )
    return df

In [ ]:
#vfu
@pf.register_dataframe_method
def step_rename_columns(df):
    """Rename columns (raw column names -> clean column names)"""
    column_names_mapping = dict(
        zip(
            data_definition["column_name_raw"].values,
            data_definition["column_name_clean"].values,
        )
    )
    df = df.rename(columns=column_names_mapping)
    return df

In [ ]:
#wf
items_raw = (pd.DataFrame()
             .step_load_raw_data()
             .step_rename_columns()
            )

In [ ]:
#pydyverse
@pf.register_dataframe_method
def step_replace_missing_values_with_nan(df, missing_values=None):
    """Replace all missing values with nan"""
    if missing_values is None:
        missing_values = ["nan", "NAN", "NaN", "none", "None", "NONE", '""', "''", ""]
    for col in df.select_dtypes(include="object").columns:
        df.loc[df[col].isin(missing_values), col] = np.nan
    return df

In [ ]:
#vfu
#TODO: write cleaner version
@pf.register_dataframe_method
def step_validate_data(df):
    """
    * replace missing with nan
    * validate unique identifiers and cardinality
    * generated dataframe of invalid entries 
    * writes data back to storage
    """
    # Remove all empty strings with nan
    unique_identifiers = list(
        data_definition[data_definition["unique_identifier"] == True]["column_name_clean"].values
    )
    validation_df = pd.DataFrame(columns=df.columns)
    validation_df["validation_error"] = None
    for unique_identifier in unique_identifiers:
        missing_df = df[unique_identifier].isna()
        duplicate_df = df[
            df[unique_identifier].isin(
                df[unique_identifier]
                .value_counts()[df[unique_identifier].value_counts() > 2]
                .index
            )
        ]
        if missing_df.shape[0] > 0:
            missing_df["validation_error"] = f"Missing {unique_identifier}"
            validation_df = pd.concat([missing_df])
        if duplicate_df.shape[0] > 0:
            duplicate_df["validation_error"] = f"Duplicate {unique_identifier}"
            validation_df = pd.concat([duplicate_df])
    return validation_df

In [ ]:
#vfu
@pf.register_dataframe_method
def step_store_invalid_data(df):
    df_name = f"{datapipe.service}-{datapipe.entity}-quality"
    azure_blob_storage.df_to_blob(
        container_name=f"{datapipe.service}-{datapipe.entity}-quality",
        blob_name=f"{datapipe.service}-{df_name}-quality-{datapipe.today}.csv",
        df=df,
    )
    return df

In [ ]:
data_definition[data_definition["unique_identifier"] == True]

In [ ]:
#wf
items_invalid = (items_raw
                 .step_replace_missing_values_with_nan()
                 .step_validate_data()
                 .step_store_invalid_data()
                )

In [ ]:
#pydyverse
@pf.register_dataframe_method
def step_remove_not_needed_columns(df):
    """Remove not needed columns"""
    to_be_removed_columns = list(
        data_definition[data_definition.to_be_removed == True][
            "column_name_clean"
        ].values
    )

    df = df.drop(columns=to_be_removed_columns)
    return df

In [ ]:
#pydyverse
@pf.register_dataframe_method
def step_remove_rows_with_invalid_ids(df):
    """Remove rows with invalid ids"""
    unique_identifiers = list(
        data_definition[data_definition.unique_identifier == True][
            "column_name_clean"
        ].values
    )
    df = df.dropna(subset=unique_identifiers)
    return df

In [ ]:
#pydyverse
@pf.register_dataframe_method
def step_remove_duplicate_rows(df, keep="last"):
    """Remove duplicate rows"""
    unique_identifiers = data_definition[data_definition.unique_identifier == True]["column_name_clean"].values
    df.drop_duplicates(subset=unique_identifiers, keep=keep, inplace=True)
    return df

In [ ]:
#pydyverse
@pf.register_dataframe_method
def step_format_dates(df, cols, date_format=None):
    for col in cols:
        if date_format:
            df[col] = pd.to_datetime(df[col], format=date_format)
        else:
            df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
    return df

In [ ]:
#pydyverse
@pf.register_dataframe_method
def step_format_round_numeric_cols(df, cols, decimal_places):
    for col in cols:
        df[col] = df[col].astype(float)
        df[col] = df[col].round(decimal_places)
    return df

In [ ]:
#pydyverse
@pf.register_dataframe_method
def step_change_datatypes(df, date_format="%Y-%m-%d %H:%M:%S"):
    """Change datatypes according to data definition"""
    _data_definition = data_definition[data_definition.to_be_removed == False]
    cols = list(
        zip(
            _data_definition["column_name_clean"].values,
            _data_definition["datatype"].values,
        )
    )
    for col in cols:
        if col[1] == "int":
            df[col[0]] = df[col[0]].astype(float)
        elif col[1] == "str":
            df[col[0]] = df[col[0]].astype(str)
        elif col[1] == "category":
            df[col[0]] = df[col[0]].astype("category")
        elif col[1] == "bool":
            df[col[0]] = df[col[0]].astype(bool)
        elif col[1] == "datetime":
            df[col[0]] = df[col[0]].astype(str)
            df[col[0]] = pd.to_datetime(df[col[0]])
        elif col[1] == "date":
            df[col[0]] = df[col[0]].astype(str)
            date_format = date_format.split(" ")[0]
            df[col[0]] = pd.to_datetime(df[col[0]])
            df[col[0]] = df[col[0]].dt.date
    return df

In [ ]:
#vfu
@pf.register_dataframe_method
def step_vfu_save_cleaned_data(df):
    azure_blob_storage.df_to_blob(
        container_name=f"{datapipe.service}-{datapipe.entity}-clean",
        blob_name=f"{datapipe.service}-{datapipe.entity}-clean.csv",
        df=df,
    )
    return df

In [ ]:
items_cleaned = (items_raw
                 .step_replace_missing_values_with_nan()
                 .step_remove_not_needed_columns()
                 .step_remove_rows_with_invalid_ids()
                 .step_remove_duplicate_rows()
                 .step_format_dates(cols=['created'])
                 .step_format_dates(cols=['modified'])
                 .step_format_round_numeric_cols(cols=['average_rating'], decimal_places=2)
                 .step_change_datatypes()
                 .step_consolidate_item_learning_levels(col='learning_level', mapping=None)
                 .step_vfu_save_cleaned_data()
                )

In [ ]:
items_cleaned.shape